## 使用davinci-003推理

In [1]:
import os
# os.environ['HTTP_PROXY']="http://172.20.137.69:8001"
# os.environ['HTTPS_PROXY']="http://172.20.137.69:8001"
# os.environ['ALL_PROXY']="socks5://172.20.137.69:8000"

# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
# from transformers import AutoTokenizer, AutoModel

# tokenizer = AutoTokenizer.from_pretrained("../models/chatglm_6b",repo_type="model", trust_remote_code=True)
# model = AutoModel.from_pretrained("../models/chatglm_6b",trust_remote_code=True).half().cuda()
# model = model.eval()

In [2]:

import logging
from collections import Counter

from utils import  load_data,getKey,write_json,mkpath

Result_Folder = "jx_result"
Err_Result_Folder = "jx_err_result"
dataset = "AQuA"
datapath = "../dataset/AQuA/AQuA_77_2.json"

In [3]:
import datetime

# 获取当前时间
current_time = datetime.datetime.now()

# 将时间格式化为文件夹名称的形式
format_time = current_time.strftime("%m%d_%H%M")

print("时间:", format_time)

if not os.path.exists(Result_Folder):
    mkpath(Result_Folder)
if not os.path.exists(f"{Result_Folder}/{dataset}"):
    mkpath(f'{Result_Folder}/{dataset}')
if not os.path.exists(Err_Result_Folder):
    mkpath(Err_Result_Folder)
if not os.path.exists(f"{Err_Result_Folder}/{dataset}"):
    mkpath(f'{Err_Result_Folder}/{dataset}')


时间: 0302_1931


In [4]:
question, answer, ids = load_data(datapath,dataset)
assert len(question) == len(answer), "输入长度不一致"
assert len(question) == len(ids),"输入长度不一致"
data_scale = len(answer)

print(f"问题个数：{len(question)}")
print(f"答案个数：{len(answer)}")

question[0],answer[0]

../dataset/AQuA/AQuA_77_2.json
问题个数：100
答案个数：100


('The height of a room to its semi-perimeter is 2:5. It costs Rs.260 to paper the walls of the room with paper 50cm wide at Rs.2 per meter allowing an area of 15 sq.m for doors and windows. The height of the room is: Answer Choices: (A) 2.6m (B) 3.9m (C) 4m (D) 4.2m (E) 4.4m',
 'C')

## 单条prompt

### 两个list分别代表201和202的所有问题的cots;每个列表是n个字典，字典内包含cots

In [5]:
from prepare_prompt import get_prompt,construct_input
from prediction_runner import basic_runner
from extracter import extract_answer,get_precision
from tqdm import tqdm


engine = "gpt-3.5-turbo-instruct"
max_length_cot = 512

correct = 0
# error_count = 0

def get_paths(prompt_id,prompt_type,path_num):
    res = []
    _, prompt = get_prompt(prompt_id=prompt_id,type=prompt_type)
    # print(prompt)
    Predict_File = f'{Result_Folder}/{dataset}/{format_time}-Type_{prompt_type}-{prompt_id}-Num_{data_scale}-{path_num}.json'
    Decoder_Error_File = f'{Err_Result_Folder}/{dataset}/{format_time}-Type_{prompt_type}-{prompt_id}-{path_num}.json'
    pred_list = []
    for idx, element in tqdm(enumerate(question)):
        inputs = construct_input(prompt, element)
        try:
            get_result, pred_list, error_msg = basic_runner(engine, inputs, max_length_cot,path_num)
            res.append({element:pred_list})
        except Exception as e:
            print("没有获取到答案！\n")
            # error_count += 1
            decode_error_data = {
                'question': question[idx],
                'answer':answer[idx]
            }
            write_json(decode_error_data, Decoder_Error_File)
            print(
                f"an error raised when predicting (question id: {ids[idx]}). "
                f"ERROR: {getattr(e.__class__, '__name__')}:{str(e)}"
            )
            continue   
        # print("pred_list:",pred_list)        
        if not get_result:
            print(
                f"not get predicted result (question id: {ids[idx]})."
                f"ERROR Message: {error_msg if error_msg else None}"
            )
            continue
        json_data = {
                        "ID": ids[idx],
                        "question": question[idx],
                        "chain-of-thought": pred_list,
                        # "pred": pred_answer,
                        "answer": answer[idx],
                        # "ans": ans
                    }    
        write_json(json_data, Predict_File)

    return res

In [6]:
def get_all_paths(prompt_id_list,path_num,prompt_type):
    res = []
    for id in prompt_id_list:
        paths = get_paths(prompt_id=id,prompt_type=prompt_type,path_num=path_num)
        res.append(paths)
    return res

def get_combined_paths(res_list:list):
    result = []
    assert len(res_list) == 2,"prompt限制两条"
    paths_1 = res_list[0]
    paths_2 = res_list[1]
    # print(len(paths_1))
    assert len(paths_1) == len(question) == len(paths_2),"path个数和问题个数不匹配"

    for j in range(len(paths_1)):
        for k,v in paths_1[j].items():
            k1,v1 = k,v
        for k,v in paths_2[j].items():
            k2,v2 = k,v
        # print('----------------')
        # print(k1)
        # print(k2)
        # print(question[j])
        # print('----------------')
        assert k1 == k2 == question[j]
        temp = {
            "question" : k1,
            "paths" : v1 + v2,
            "answer" : answer[j]
        }
            
        result.append(temp)
    return result

# test_a = [[{'a':[1]},{'b':[2]}],[{'a':[3]},{'b':[4]}]]
# get_combined_paths(test_a)
        

In [18]:
import json 
f1 = []
f2 = []
with open('./jx_result/AQuA/0228_1554-Type_1-201-Num_100-5.json','r',encoding='utf-8') as f11:
    f1 = json.load(f11)
with open('./jx_result/AQuA/0228_1554-Type_1-202-Num_100-5.json','r',encoding='utf-8') as f22:
    f2 = json.load(f22)    
len(f1),len(f2)

(100, 100)

In [9]:
# f1[0]

In [19]:
t1 = []
t2 = []
for i in range(len(f1)):
    temp1 = {}
    temp2 = {}
    q1 = f1[i]['question']
    v1 = f1[i]['chain-of-thought']
    temp1[q1] = v1
    t1.append(temp1)
    q2 = f2[i]['question']
    v2 = f2[i]['chain-of-thought']
    temp2[q2] = v2
    t2.append(temp2)
all_paths = []
all_paths.append(t1)
all_paths.append(t2)

In [7]:
prompt_id_list = [201,202]
path_num = 5
prompt_type = 1



all_paths = get_all_paths(prompt_id_list=prompt_id_list,path_num=path_num,prompt_type=prompt_type)

res_paths = get_combined_paths(all_paths)

100it [09:03,  5.44s/it]
100it [09:16,  5.57s/it]


In [8]:
assert len(prompt_id_list) == 2
ids = prompt_id_list[0] +prompt_id_list[1]

temp_p = f'{Result_Folder}/{dataset}/temp-{format_time}-Type_{prompt_type}-{ids}-Num_{data_scale}-{path_num}.json'
# temp_p = "./jx_result/AQuA/temp-0228_1554-Type_1-201-Num_100_0-5.json"
write_json(res_paths,path=temp_p)

## 使用chatGLM抽取答案

### 准备数据

In [ ]:
import json
# temp_p = "jx_result/AQuA/temp-1031_1639-Type_1-403-Num_77-5.json"

with open(temp_p,'r',encoding='utf-8') as f:
    data = json.load(f)
len(data),data[0]

### 抽取答案

In [ ]:
def chatGLM(text,history):
    res,history  = model.chat(tokenizer,text,history)
#     print(res)
    return res

In [ ]:
# induce = "\nPlease extract the  option from the text.\n Reply format should be the answer is xxx.\nJust replay option.\n Example: The answer is (A) xx."

In [ ]:
# a = '[Text]:\n' + data[9]['paths'][5] + induce 
print(a)

In [ ]:
# chatGLM(a,[])

In [ ]:
from tqdm import tqdm

induce = "\nPlease extract the answer option from the text.\n Reply format should be the answer is xxx.\nJust replay option.\n Example: The answer is (A) xx."
for i in tqdm(range(len(data))):
    cots = data[i]['paths']
    ans = []
    for cot in cots:
        induce_prompt = '[Text]:\n' + cot + induce 
#         print(induce_prompt)
        try:
            res = chatGLM(induce_prompt,[])
            ans.append(res)
        except Exception as e:
            print('出错了！')
            print(e)
    data[i]['answers'] = ans    
   

In [ ]:
import copy

raw_data = copy.deepcopy(data)

In [ ]:
data[50]['answers'],data[50]['answer']

In [ ]:
import re

def extract_option(text:str):
    pattern = r'A|B|C|D|E'
    match = re.search(pattern=pattern,string=text)
    if match:
        answer = str(match.group())
    else:
        answer = None
    return answer


def extract_answer(text: str):
    count = 0
    pattern_1 = r'\d+(?:\.\d+)?'
    # 尝试直接从cot中抽取答案
    answer = extract_option(text)
    # 如果抽不出来，则将其再次输入chatGLM-6b,得到新的text
    if answer == None:
        count += 1
        answer = 'E'
    return answer,count

In [ ]:
from tqdm import  tqdm
    
for i in tqdm(range(len(data))):
    ans = []
    answers = data[i]['answers']
    for item in answers:
        answer,count = extract_answer(item)
        ans.append(answer)
    data[i]['answers'] = ans
count

In [ ]:
from utils import write_json

prompt_id_list = [201,202]

assert len(prompt_id_list) == 2
ids = prompt_id_list[0] +prompt_id_list[1]

# res_p = f'{Result_Folder}/{dataset}/res-{format_time}-Type_{prompt_type}-{ids}-Num_{data_scale}-{path_num}.json'
res_p = "jx_result/AQuA/res-1031_1954-Type_1-403-Num_77-5.json"

write_json(data,path=res_p)

## 筛选器

In [ ]:
import json

with open(res_p,'r',encoding='utf-8') as f:
    res_data = json.load(f)
len(res_data),res_data[0]

In [ ]:
from collections import Counter

for item in data:
    answers = item['answers']
    jx = Counter(answers).most_common(1)[0][0]
    item['pred_answer'] = jx

data[0]

In [ ]:
c1 = 0
c2 = 0
for item in data:
    pred = item['pred_answer']
    gold = item['answer']
    if pred:
        if str(pred) == str(gold):
            item['ans'] = True
            c1 += 1
        else:
            item['ans'] = False
            c2 += 1
    else:
        c2 += 1
assert c1 + c2 == len(data)
c1,c2



In [ ]:
p = res_p.replace('res-','ses-')

write_json(res_data,p)

In [ ]:
28/77

In [ ]:
# import json

# path = './jx_result/gsm8k/1018_1021-Type_1-303-Num_100-0.json'

# data = []
# with open(path,'r') as f:
#    for line in f:
#         data.append(line)
# len(data)

# count = 0
# pos = 0
# neg = 0
# for item in data:
#     if "\"ans\":" in item:
#         count+=1;
#         if "true" in item:
#             pos+=1
#         else:
#             neg+=1
# assert count == pos + neg,"统计错误"
# print(count,pos)
# print(pos/count)

In [ ]:
# print('Brenda will have a total of $450 after 3 years. This can be calculated using the formula for simple interest: \n\nTotal Amount = Principal Amount + Interest Earned \n\nTotal Amount = 300 + (300 x 0.75 x 3) \n\nTotal Amount = 300 + 225 \n\nTotal Amount = $450')

In [ ]:
(203+199+198)/900

In [ ]:
(0.397+0.354+0.5)/3

In [ ]:
(184+33)/300

In [ ]:
c1 = 2
c2 = 3

print("计算完毕，准确率：{},正确个数：{}，错误个数：{}".format(c1/(c1+c2),c1,c2))